In [2]:
from scipy.stats import binom
import random
import numpy as np
import matplotlib.pyplot as plt


In [3]:
female_mean = 68
female_dev  = 3.5
male_mean   = 70
male_dev    = 4

female_distribution = np.random.normal(female_mean, female_dev, 1000)
male_distribution   = np.random.normal(male_mean,   male_dev,   1000)

In [11]:
# plt.figure(figsize=(21,13))
# female_count, female_bins, female_ignored = plt.hist(female_distribution, 30, density=True)
# plt.plot(female_bins, 1/(female_dev * np.sqrt(2 * np.pi)) *
#           np.exp( - (female_bins - female_mean)**2 / (2 * female_dev**2) ),
#           linewidth=2, color='g')
# male_count, male_bins, male_ignored = plt.hist(male_distribution, 30, density=True)
# plt.plot(male_bins, 1/(male_dev * np.sqrt(2 * np.pi)) *
#           np.exp( - (male_bins - male_mean)**2 / (2 * male_dev**2) ),
#           linewidth=2, color='r')
# plt.show()

In [5]:
female_distribution = sorted(female_distribution)
male_distribution   = sorted(male_distribution)
female_index = 0
male_index   = 0
height_array = []
female_height_array = []
male_height_array   = []
female_prob_array   = []
male_prob_array     = []
height_range = np.arange(round(female_distribution[0], 1), round(male_distribution[len(male_distribution)-1]+1, 1), 1)

for i in height_range:
    female_count = 0
    male_count   = 0
    while female_index < len(female_distribution) and female_distribution[female_index] <= i:
        female_count += 1
        female_index += 1
    while male_index < len(male_distribution) and male_distribution[male_index] <= i:
        male_count += 1
        male_index += 1
    height_array.append(i)
    female_height_array.append(female_count)
    male_height_array  .append(male_count)
    if(female_count + male_count == 0):
        female_prob_array.append(0)
        male_prob_array  .append(0)
    else:
        female_prob_array.append(female_count/(female_count + male_count))
        male_prob_array  .append(male_count/(female_count + male_count))
plt.figure(figsize=(21,13))
plt.plot(height_array, female_prob_array, color="g")
plt.plot(height_array, male_prob_array, color="r")

In [6]:
def get_prediction(height, prob_treshold):
    index = 0
    while height_array[index] < height:
        index += 1
    index -= 1
    if female_prob_array[index] >= prob_treshold:
        return 'f'
    else:
        return 'm'

In [7]:
def get_tpr(prob_treshold):
    false_positive = 0
    true_positive  = 0
    for element in female_distribution:
        if(get_prediction(element, prob_treshold) == 'f'):
            true_positive += 1
        else:
            false_positive += 1
#     for element in male_distribution:
#         if(get_prediction(element, prob_treshold) == 'm'):
#             true_positive += 1
#         else:
#             false_positive += 1
#     print("true positives rate: "+str(true_positive/(len(female_distribution) + len(male_distribution))*100)+
#           "%, false positive rate: "+str(false_positive/(len(female_distribution) + len(male_distribution))*100)+"%.")
#     return true_positive/(len(female_distribution) + len(male_distribution))
    return true_positive/len(female_distribution)

In [8]:
# https://www.medcalc.org/manual/roc-curves.php

min_value = min([min(female_distribution),min(male_distribution)])
max_value = max([max(female_distribution),max(male_distribution)])
value_range = max_value - min_value
tp_array = []
fp_array = []
tn_array = []
fn_array = []

tpr_array = []
fpr_array = []

def get_prediction2(height, treshold_value):
    if height < treshold_value:
        return 'f'
    else:
        return 'm'

def get_classification_stats(treshold): #treshold in range [0,1]
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    treshold_value = min_value + (treshold * value_range)
    for element in female_distribution:
        if(get_prediction2(element, treshold_value) == 'f'):
            tp += 1
        else:
            fn += 1
    for element in male_distribution:
        if(get_prediction2(element, treshold_value) == 'm'):
            tn += 1
        else:
            fp += 1
    tp_array.append(tp)
    fp_array.append(fp)
    tn_array.append(tn)
    fn_array.append(fn)
    
    tpr_array.append(tp/(tp+fn))
    fpr_array.append(1-(tn/(fp+tn)))

In [9]:
treshold_range = np.arange(0, 1, 0.01)
for treshold in treshold_range:
    get_classification_stats(treshold)

plt.figure(figsize=(13,13))
plt.plot(fpr_array, tpr_array)

In [10]:
cumulated_cost = []
minimum_cost_index = 0
for i in range(0, len(fp_array)):
    cumulated_cost.append(2*fp_array[i] + 3*fn_array[i])
    if cumulated_cost[minimum_cost_index] > cumulated_cost[len(cumulated_cost)-1]:
        minimum_cost_index = len(cumulated_cost)-1

minimum_cost_treshold_value = min_value + (treshold_range[minimum_cost_index] * value_range)
print(minimum_cost_index)
print(minimum_cost_treshold_value)

plt.figure(figsize=(13,13))
plt.plot(treshold_range, cumulated_cost)


61
71.99707134288478
